In [20]:
config = {
    "pretrained_model": "bert-base-cased",
    "tokenizer": "bert-base-cased",
    "max_seq_length": 256,
    "batch_size": 16,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "0,1,2,3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/bert_cased_512_biocaster_4cate_20200903/",
    "patience": 4,
}

In [21]:
import sys
import pandas as pd
sys.path.append("../")

In [22]:
train_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/train.csv")
dev_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/dev.csv")
test_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/test.csv")
train_bt_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/train_bt.csv")

In [14]:
label_mapping = {
    0:"0001",
    1:"0010",
    2:"0100",
    3:"1000",
}
train_set.labels=train_set.labels.apply(lambda x:label_mapping[x])
train_set.docs = train_set.docs.apply(lambda x: x.replace("\n",". ").replace("\r",""))
dev_set.docs = dev_set.docs.apply(lambda x: x.replace("\n",". ").replace("\r",""))
test_set.docs = test_set.docs.apply(lambda x: x.replace("\n",". ").replace("\r",""))

train_set.head()

,Unnamed: 0,index,docs,labels,source,flag
0,0,993,". ONCHOCERCIASIS, DRUG RESISTANCE - GHANA. Dat...",1000,Biocaster,train
1,1,467,. Avian influenza situation in Thailand ? up...,1000,Biocaster,train
2,2,135,. TB spreading fast. . Published on Jul 5...,0001,Biocaster,train
3,3,204,. MEASLES - UKRAINE (KHMELNIZKIY) (02) *****...,1000,Biocaster,train
4,4,760,. Romania confirms deadly bird flu in six vi...,0100,Biocaster,train


In [15]:
dev_set.labels=dev_set.labels.apply(lambda x:label_mapping[x])
test_set.labels=test_set.labels.apply(lambda x:label_mapping[x])


In [19]:
train_set.to_csv("../covid19-classification/hedwig-data/datasets/Biocaster4C/train.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")
dev_set.to_csv("../covid19-classification/hedwig-data/datasets/Biocaster4C/dev.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")
test_set.to_csv("../covid19-classification/hedwig-data/datasets/Biocaster4C/test.tsv",
                columns=["labels","docs"],
                header=False,
                index=False,
                sep="\t")

In [ ]:
len(train_bt_set.index)

In [ ]:
def run_bert(train_set,dev_set,test_set, lans, sample_rate):
    print("train/dev/test numbers: ", len(train_set.index),len(dev_set.index),len(test_set.index))
    
    from beta_nlp.models.bert_cls import BertModel
    from beta_nlp.utils.common import save_to_csv
    
    cls = BertModel(config)
    cls.train(train_set,dev_set)
    
    result = cls.test(test_set)
    result["lans"] = lans
    result["sample_rate"] = sample_rate
    save_to_csv(result,result_file="../results/20201001_backtrans.csv")
    
def train_sample(train_bt_set,lan_set,sample_rate):
    """ sample back translation document samples
    """
    train_set = train_bt_set[train_bt_set["lan"]=="en"]
    for lan in lan_set:
        if lan =="en":
            continue
        train_set_sample = train_bt_set[train_bt_set["lan"]==lan]
        from sklearn.utils import shuffle
        train_set_sample = shuffle(train_set_sample).reset_index()
        sample_n = int(len(train_set_sample)*sample_rate)
        train_set_sample = train_set_sample.iloc[:sample_n]
        train_set = train_set.append(train_set_sample)
    return train_set

### original bert

In [ ]:
# repeat = 5
# for i in range(repeat):
#     run_bert(train_set,dev_set,test_set, lans=["en"], sample_rate=1)

### single language backtranslation

In [ ]:
repeat = 2
lang_list = ["ar","zh","fr","es","pt","ru"]
sample_rate_list = [0.1,0.25,0.5,0.75,1]
for lang in lang_list:
    lan_set = ["en"]
    lan_set.append(lang)
    for sample_rate in sample_rate_list:
        for i in range(repeat):
            train_set = train_sample(train_bt_set,lan_set,sample_rate)
            run_bert(train_set, dev_set, test_set, lans=lan_set, sample_rate=sample_rate)

### six languages backtranslation

In [ ]:
repeat = 2
lang_list = ["ar","zh","fr","es","pt","ru"]
sample_rate_list = [0.1,0.25,0.5,0.75,1]
lan_set = lang_list
lan_set.append(lang)
for sample_rate in sample_rate_list:
    for i in range(repeat):
        train_set = train_sample(train_bt_set,lan_set,sample_rate)
        run_bert(train_set, dev_set, test_set, lans=lan_set, sample_rate=sample_rate)

### two languages backtranslation

In [ ]:
repeat = 2
lang_list = ["ar","zh","fr","es","pt","ru"]
sample_rate_list = [0.1,0.25,0.5,0.75,1]
for i in range(len(lang_list)-1):
    for j in range(i+1,len(lang_list)):
        lan_set = ["en"]+[lang_list[i]]+[lang_list[j]]
        for sample_rate in sample_rate_list:
            for i in range(repeat):
                train_set = train_sample(train_bt_set,lan_set,sample_rate)
                run_bert(train_set, dev_set, test_set, lans=lan_set, sample_rate=sample_rate)

In [10]:
train_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/train.csv")
dev_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/dev.csv")
test_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/test.csv")
train_bt_set = pd.read_csv("/home/zm324/workspace/doc_cls/datasets/biocaster/train_bt_full.csv")

In [11]:
len(train_bt_set.index)

5614

In [12]:
def run_bert(train_set,dev_set,test_set, lans, sample_rate):
    print("train/dev/test numbers: ", len(train_set.index),len(dev_set.index),len(test_set.index))
    
    from beta_nlp.models.bert_cls import BertModel
    from beta_nlp.utils.common import save_to_csv
    
    cls = BertModel(config)
    cls.train(train_set,dev_set)
    
    result = cls.test(test_set)
    result["lans"] = lans
    result["sample_rate"] = sample_rate
    save_to_csv(result,result_file="../results/20201001_backtrans_full.csv")
    
def train_sample(train_bt_set,lan_set,sample_rate):
    """ sample back translation document samples
    """
    train_set = train_bt_set[train_bt_set["lan"]=="en"]
    for lan in lan_set:
        if lan =="en":
            continue
        train_set_sample = train_bt_set[train_bt_set["lan"]==lan]
        from sklearn.utils import shuffle
        train_set_sample = shuffle(train_set_sample).reset_index()
        sample_n = int(len(train_set_sample)*sample_rate)
        train_set_sample = train_set_sample.iloc[:sample_n]
        train_set = train_set.append(train_set_sample)
    return train_set

### single language backtranslation

In [13]:
repeat = 2
lang_list = ["ar","zh","fr","es","pt","ru"]
sample_rate_list = [0.1,0.25,0.5,0.75,1]
for lang in lang_list:
    lan_set = ["en"]
    lan_set.append(lang)
    for sample_rate in sample_rate_list:
        for i in range(repeat):
            train_set = train_sample(train_bt_set,lan_set,sample_rate)
            run_bert(train_set, dev_set, test_set, lans=lan_set, sample_rate=sample_rate)

train/dev/test numbers:  882 100 101


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (


Execute [train_an_epoch] method costing 11492.74 ms
[Epoch 0] loss: 65.05565077066422



Execute [eval] method costing 518.83 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.73     |
|  1 | macro_precision | 0.428191 |
|  2 | macro_recall    | 0.298387 |
|  3 | macro_f1        | 0.289156 |
|  4 | micro_precision | 0.73     |
|  5 | micro_recall    | 0.73     |
|  6 | micro_f1        | 0.73     |
+----+-----------------+----------+
--------------------------------------------------------------------------------


...


Execute [train_an_epoch] method costing 20686.71 ms
[Epoch 8] loss: 4.455283866846003



Execute [eval] method costing 523.19 ms



Execute [train_an_epoch] method costing 20708.38 ms
[Epoch 9] loss: 2.060664892254863



Execute [eval] method costing 531.18 ms




Execute [eval] method costing 891.19 ms
--------------------------------------------------------------------------------
+----+-----------------+---------------------+
|    | keys            | values              |
|----+-----------------+---------------------|
|  0 | accuracy        | 0.7722772277227723  |
|  1 | macro_precision | 0.46759259259259256 |
|  2 | macro_recall    | 0.47488839285714285 |
|  3 | macro_f1        | 0.46676377118644063 |
|  4 | micro_precision | 0.7722772277227723  |
|  5 | micro_recall    | 0.7722772277227723  |
|  6 | micro_f1        | 0.7722772277227723  |
|  7 | lans            | ['en', 'ru']        |
|  8 | sample_rate     | 1                   |
+----+-----------------+---------------------+
--------------------------------------------------------------------------------
../results/20201001_backtrans_full.csv  already exists, appending result to it


### six languages backtranslation

In [14]:
repeat = 2
lang_list = ["ar","zh","fr","es","pt","ru"]
sample_rate_list = [0.1,0.25,0.5,0.75,1]
lan_set = lang_list
lan_set.append(lang)
for sample_rate in sample_rate_list:
    for i in range(repeat):
        train_set = train_sample(train_bt_set,lan_set,sample_rate)
        run_bert(train_set, dev_set, test_set, lans=lan_set, sample_rate=sample_rate)

train/dev/test numbers:  1362 100 101


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (


Execute [train_an_epoch] method costing 17663.99 ms
[Epoch 0] loss: 94.96821820735931



Execute [eval] method costing 526.82 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.77     |
|  1 | macro_precision | 0.380864 |
|  2 | macro_recall    | 0.365311 |
|  3 | macro_f1        | 0.372692 |
|  4 | micro_precision | 0.77     |
|  5 | micro_recall    | 0.77     |
|  6 | micro_f1        | 0.77     |
+----+-----------------+----------+
--------------------------------------------------------------------------------


...


Execute [train_an_epoch] method costing 82144.61 ms
[Epoch 7] loss: 4.351748419307114



Execute [eval] method costing 524.09 ms



Execute [train_an_epoch] method costing 82572.01 ms
[Epoch 8] loss: 5.4412772489959025



Execute [eval] method costing 523.50 ms
Early Stopping. Epoch: 8, best_valid_metric (macro_f1): 0.5637357178340785




Execute [eval] method costing 526.64 ms
--------------------------------------------------------------------------------
+----+-----------------+--------------------------------------------+
|    | keys            | values                                     |
|----+-----------------+--------------------------------------------|
|  0 | accuracy        | 0.7920792079207921                         |
|  1 | macro_precision | 0.5066468253968254                         |
|  2 | macro_recall    | 0.52734375                                 |
|  3 | macro_f1        | 0.507032279314888                          |
|  4 | micro_precision | 0.7920792079207921                         |
|  5 | micro_recall    | 0.7920792079207921                         |
|  6 | micro_f1        | 0.7920792079207921                         |
|  7 | lans            | ['ar', 'zh', 'fr', 'es', 'pt', 'ru', 'ru'] |
|  8 | sample_rate     | 1                                          |
+----+-----------------+--------------

### two languages backtranslation

In [ ]:
repeat = 2
lang_list = ["ar","zh","fr","es","pt","ru"]
sample_rate_list = [0.1,0.25,0.5,0.75,1]
for i in range(len(lang_list)-1):
    for j in range(i+1,len(lang_list)):
        lan_set = ["en"]+[lang_list[i]]+[lang_list[j]]
        for sample_rate in sample_rate_list:
            for i in range(repeat):
                train_set = train_sample(train_bt_set,lan_set,sample_rate)
                run_bert(train_set, dev_set, test_set, lans=lan_set, sample_rate=sample_rate)

train/dev/test numbers:  962 100 101


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (


Execute [train_an_epoch] method costing 12580.45 ms
[Epoch 0] loss: 69.58321756124496



Execute [eval] method costing 524.25 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.69     |
|  1 | macro_precision | 0.420918 |
|  2 | macro_recall    | 0.266129 |
|  3 | macro_f1        | 0.233333 |
|  4 | micro_precision | 0.69     |
|  5 | micro_recall    | 0.69     |
|  6 | micro_f1        | 0.69     |
+----+-----------------+----------+
--------------------------------------------------------------------------------


...


Execute [eval] method costing 528.97 ms



Execute [train_an_epoch] method costing 15440.10 ms
[Epoch 7] loss: 7.399456010549329



Execute [eval] method costing 530.44 ms



Execute [train_an_epoch] method costing 15804.22 ms
[Epoch 8] loss: 7.000572843942791



Execute [eval] method costing 527.45 ms
